In [7]:
import cv2
from deepface import DeepFace

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        margin = 25
        x1 = max(0, x - margin)
        y1 = max(0, y - margin)
        x2 = min(frame.shape[1], x + w + margin)
        y2 = min(frame.shape[0], y + h + margin)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Process every 5th frame
        if frame_count % 5 == 0:
            try:
                face_crop = frame[y1:y2, x1:x2]
                face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

                result = DeepFace.analyze(
                    img_path=face_rgb,
                    actions=["emotion"],
                    enforce_detection=False
                )

                if isinstance(result, list):
                    result = result[0]

                emotion = result.get("dominant_emotion", "Unknown")

            except Exception as e:
                emotion = "Unknown"

        # Display emotion
        cv2.putText(frame, f"Emotion: {emotion}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    frame_count += 1
    cv2.imshow("Face Emotion", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 